In [1]:
# --- 1. Mount Drive & Set Up Project ---
from google.colab import drive
import os
drive.mount('/content/drive')
!unzip -o /content/drive/MyDrive/powershell-sentinel-main.zip -d /content/
%cd /content/powershell-sentinel-main

# --- 2. Authenticate with Hugging Face ---
# This step is now CRITICAL for downloading Llama 3
!huggingface-cli login
# Paste your Hugging Face 'read' token when prompted

# --- 3. Definitive Library Installation ---
# This is the final, corrected installation command for a stable A100 environment.
# It includes 'triton', a critical dependency for bitsandbytes.
!pip install -U "transformers==4.41.2" "datasets==2.19.1" "accelerate==0.30.1" "peft==0.10.0" "trl==0.8.6" "bitsandbytes==0.43.1" "triton"

print("\n\n✅ ✅ ✅ MASTER SETUP COMPLETE. You are ready to train. ✅ ✅ ✅")

Mounted at /content/drive
Archive:  /content/drive/MyDrive/powershell-sentinel-main.zip
1fbaf135538551d92f958d39f8eefba66f855a44
   creating: /content/powershell-sentinel-main/
  inflating: /content/powershell-sentinel-main/.gitignore  
  inflating: /content/powershell-sentinel-main/README.md  
   creating: /content/powershell-sentinel-main/data/
   creating: /content/powershell-sentinel-main/data/generated/
  inflating: /content/powershell-sentinel-main/data/generated/training_data_v0.json  
  inflating: /content/powershell-sentinel-main/data/generated/training_data_v0_clean.json  
   creating: /content/powershell-sentinel-main/data/interim/
   creating: /content/powershell-sentinel-main/data/interim/curating_logs/
  inflating: /content/powershell-sentinel-main/data/interim/curating_logs/uncurated_for_review.json  
   creating: /content/powershell-sentinel-main/data/interim/delta_logs/
  inflating: /content/powershell-sentinel-main/data/interim/delta_logs/PS-001.json  
 extracting: /c

In [2]:
# --- 3. Definitive Library Installation ---
# This is the final, corrected installation command for a stable A100 environment.
# It forcefully uninstalls conflicting libraries and pins Triton to a known-good version.
!pip uninstall bitsandbytes triton -y
!pip install -U "transformers==4.41.2" "datasets==2.19.1" "accelerate==0.30.1" "peft==0.10.0" "trl==0.8.6" "bitsandbytes==0.43.1" "triton==2.3.1"

Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl.metadata (2.2 kB)
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 115.8 

In [4]:
# --- 4. THE SMOKE TEST ---
import json

print("--- Step A: Creating Smoke Test Data ---")
# Create a tiny 3-sample training and validation set
full_data_path = '/content/powershell-sentinel-main/data/generated/training_data_v0_clean.json'
smoke_train_path = '/content/powershell-sentinel-main/data/generated/smoke_test_train.json'
smoke_val_path = '/content/powershell-sentinel-main/scripts/prompt_engineering/smoke_test_val.json'

with open(full_data_path, 'r') as f:
    data = json.load(f)
with open(smoke_train_path, 'w') as f:
    json.dump(data[:3], f)
with open(smoke_val_path, 'w') as f:
    json.dump(data[3:6], f)

print("--- Step B: Running a 10-Step Smoke Test Training ---")
!python -m powershell_sentinel.train \
    --model_name meta-llama/Meta-Llama-3-8B-Instruct \
    --train_dataset {smoke_train_path} \
    --preflight_train_dataset {smoke_train_path} \
    --output_dir models/smoke_test_model \
    --test_dataset data/sets/test_set_v0.json \
    --learning_rate 2e-5 \
    --lora_rank 16 \
    --max_steps 10

print("\n--- Step C: Running a 3-Sample Smoke Test Evaluation ---")
!python -m powershell_sentinel.evaluate \
    --base_model_path meta-llama/Meta-Llama-3-8B-Instruct \
    --model_path models/smoke_test_model/final_checkpoint \
    --test_set_path {smoke_val_path}

print("\n\n✅ ✅ ✅ SMOKE TEST COMPLETE. If there were no errors, you are safe to proceed. ✅ ✅ ✅")

--- Step A: Creating Smoke Test Data ---
--- Step B: Running a 10-Step Smoke Test Training ---
2025-08-12 03:13:33.513068: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-12 03:13:33.531428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754968413.553134    5588 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754968413.559806    5588 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754968413.577039    5588 computatio

In [24]:
# --- 5. DATASET CLEANING/PARTITIONING ---
# --- Step 1: Clean Slate ---
!rm -rf /content/powershell-sentinel-main/data/sets/*
!rm -rf /content/powershell-sentinel-main/scripts/prompt_engineering/mini_*.json
!rm -f /content/powershell-sentinel-main/data/generated/training_data_v0_clean.json

# --- Step 2: Regenerate All Datasets Correctly ---
!python -m scripts.deduplicate_dataset
!python -m scripts.partition_dataset

print("\n\n✅ DATASETS REGENERATED. LEAKAGE IS NOW IMPOSSIBLE. ✅")

/usr/bin/python3: Error while finding module specification for 'scripts.deduplicate_dataset' (ModuleNotFoundError: No module named 'scripts')
/usr/bin/python3: Error while finding module specification for 'scripts.partition_dataset' (ModuleNotFoundError: No module named 'scripts')


✅ DATASETS REGENERATED. LEAKAGE IS NOW IMPOSSIBLE. ✅


In [13]:
# --- 6. FINAL MODEL TRAINING ---
import json
import os

# Use the NEWLY CREATED training_set_v0.json from the partition script
final_train_path = '/content/powershell-sentinel-main/data/sets/training_set_v0.json'
with open(final_train_path, 'r') as f:
    num_samples = len(json.load(f))

effective_batch_size = 4
steps_per_epoch = num_samples // effective_batch_size
max_steps_for_2_epochs = steps_per_epoch * 2

print(f"Final training set has {num_samples} samples.")
print(f"Training for 2 epochs will take exactly {max_steps_for_2_epochs} steps.")

# Define paths for the command
output_dir = "models/final_model"
drive_backup_path = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/final_model_adapters.zip"

# --- THE CORRECTED COMMAND ---
# This command is now simplified and matches the final train.py script.
!python -m powershell_sentinel.train \
    --model_name meta-llama/Meta-Llama-3-8B-Instruct \
    --train_dataset {final_train_path} \
    --output_dir {output_dir} \
    --test_dataset data/sets/test_set_v0.json \
    --learning_rate 2e-5 \
    --lora_rank 16 \
    --max_steps {max_steps_for_2_epochs}

# --- Immediately back up the final model to Google Drive ---
!mkdir -p /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable
!zip -r {drive_backup_path} /content/powershell-sentinel-main/{output_dir}

print(f"\n\n✅ ✅ ✅ FINAL TRAINING COMPLETE AND BACKED UP TO DRIVE: {drive_backup_path} ✅ ✅ ✅")

Final training set has 5117 samples.
Training for 2 epochs will take exactly 2558 steps.
2025-08-11 04:37:17.379820: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 04:37:17.398496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754887037.420206   13904 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754887037.426785   13904 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754887037.444030   13904 computation_plac

In [14]:
# --- 7. FINAL MODEL EVALUATION ---
# This will produce the final results table for your dissertation.
report_filename = "final_evaluation_report.md"
local_report_path = f"/content/powershell-sentinel-main/{report_filename}"
drive_report_path = f"/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/{report_filename}"

!python -m powershell_sentinel.evaluate \
    --base_model_path meta-llama/Meta-Llama-3-8B-Instruct \
    --model_path models/final_model/final_checkpoint \
    --test_set_path data/sets/test_set_v0.json > {local_report_path}

# --- Immediately back up the final report to Google Drive ---
!cp {local_report_path} {drive_report_path}

# Print the report to the screen as well
!cat {local_report_path}

print(f"\n\n✅ ✅ ✅ FINAL EVALUATION COMPLETE AND REPORT BACKED UP TO DRIVE. ✅ ✅ ✅")

2025-08-11 05:49:57.443821: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 05:49:57.462662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754891397.484259   32541 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754891397.490836   32541 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754891397.508490   32541 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [15]:
# --- (Obsolete) 8. CREATE FINAL GGUF DELIVERABLE ---
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import os
import sys
import site

# --- Step 1: Define Paths ---
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
adapter_path = "/content/powershell-sentinel-main/models/final_model/final_checkpoint"
merged_model_path = "/content/merged_final_model"
gguf_output_path = "/content/powershell_sentinel_llama3_q4km.gguf"
drive_gguf_path = f"/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/powershell_sentinel_llama3_q4km.gguf"

# --- Step 2: Load and Merge the Model (No Changes Here) ---
compute_dtype = getattr(torch, "bfloat16")
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=compute_dtype)

print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, quantization_config=quant_config, device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print("Loading adapters and merging...")
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.merge_and_unload()
print("Model merged successfully.")

print(f"Saving merged model to {merged_model_path}...")
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

# --- Step 3: Forceful Installation of llama-cpp-python from Source ---
# This is the definitive fix. It guarantees the convert.py script is available.
print("\nInstalling llama-cpp-python from source...")
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --no-cache-dir --force-reinstall llama-cpp-python -q

# --- Step 4: GGUF Conversion (No Changes Here, but it will now work) ---
# We use a more robust method to find the script, just in case
CONVERT_SCRIPT_PATH = None
for path in site.getsitepackages():
    if os.path.exists(os.path.join(path, 'llama_cpp', 'convert.py')):
        CONVERT_SCRIPT_PATH = os.path.join(path, 'llama_cpp', 'convert.py')
        break

if CONVERT_SCRIPT_PATH:
    print(f"Found conversion script at {CONVERT_SCRIPT_PATH}")
    print("Starting GGUF conversion...")

    !python {CONVERT_SCRIPT_PATH} {merged_model_path} --outfile {gguf_output_path} --outtype q4_k_m

    print(f"\n\n✅ GGUF conversion complete! File at: {gguf_output_path}")

    # --- Step 5: Backup the final GGUF to Google Drive ---
    !cp {gguf_output_path} {drive_gguf_path}
    print(f"✅ Final GGUF model backed up to: {drive_gguf_path}")
else:
    print("❌ CRITICAL ERROR: Could not find llama.cpp convert.py script even after source installation.")

Loading base model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading adapters and merging...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Model merged successfully.
Saving merged model to /content/merged_final_model...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 MB 48.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
❌ Could not find llama.cpp convert.py script.


In [3]:
# --- (Current) 8. CREATE FINAL GGUF DELIVERABLE ---
# ==============================================================================
# DEFINITIVE SCRIPT: CREATE & SAVE ALL GGUF MODELS (Consolidated Version)
# ==============================================================================

# --- 1. SETUP AND PATH DEFINITIONS ---
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
from google.colab import drive

# THE CRITICAL FIX: Mount Google Drive at the very beginning to ensure it's connected.
print("--- Mounting Google Drive ---")
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully.")

# Define all necessary paths
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
drive_backup_path = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/final_model_adapters.zip"
local_restore_dir = "/content/restored_model_temp"
merged_model_path = "/content/merged_model_final" # Absolute path
llama_cpp_path = "/content/llama.cpp"

# Define the base name for the GGUF files
gguf_base_name = "powershell_sentinel_llama3"
intermediate_gguf_path = f"/content/{gguf_base_name}_f16.gguf"

# Define the list of quantization "flavours" to create
quantization_types = [
    "Q8_0", "Q6_K", "Q5_K_M", "Q4_K_M", "Q3_K_M", "Q2_K"
]
print(f"\n✅ Will generate {len(quantization_types)} GGUF models: {', '.join(quantization_types)}")

# --- 2. THE "CLEAN MERGE" WORKFLOW ---
print(f"\nRestoring fine-tuned model adapters from: {drive_backup_path}")
# Ensure the restore directory is clean for idempotent runs
!rm -rf {local_restore_dir}
!mkdir -p {local_restore_dir}
!unzip -o -q "{drive_backup_path}" -d {local_restore_dir}
adapter_path = next((os.path.join(root, d) for root, dirs, _ in os.walk(local_restore_dir) for d in dirs if "final_checkpoint" in d), None)
if not adapter_path:
    raise FileNotFoundError("Could not find 'final_checkpoint' directory.")
print(f"Found adapter checkpoint at: {adapter_path}")

print("\nLoading base model in bfloat16 for a clean merge...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
print("✅ Base model loaded successfully.")

print("\nLoading PEFT adapters and merging...")
model = PeftModel.from_pretrained(base_model, adapter_path).merge_and_unload()
print("✅ Model and adapters merged successfully.")

print(f"\nSaving the clean, high-precision merged model to: {merged_model_path}")
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
del model, base_model
torch.cuda.empty_cache()
print("✅ Merged model saved and GPU memory cleaned.")

# --- 3. CLONE LLAMA.CPP AND INSTALL REQUIREMENTS ---
print("\nCloning llama.cpp repository and installing dependencies...")
# Only clone if the directory doesn't exist to save time on reruns
if not os.path.exists(llama_cpp_path):
    !git clone https://github.com/ggerganov/llama.cpp.git {llama_cpp_path}
!pip install -r {os.path.join(llama_cpp_path, 'requirements.txt')} -q
print("✅ llama.cpp is ready and dependencies are installed.")

# --- 4. GGUF CREATION - THE CORRECT TWO-STEP PROCESS ---
%cd {llama_cpp_path}
print(f"\nChanged working directory to: {os.getcwd()}")

# STEP 4A: Convert the clean HF model to a base f16 GGUF file.
print("\n--- Step 4A: Converting to initial f16 GGUF format ---")
!python convert_hf_to_gguf.py "{merged_model_path}" \
  --outfile "{intermediate_gguf_path}" \
  --outtype f16
if not os.path.exists(intermediate_gguf_path):
    raise RuntimeError("Failed to create the intermediate f16 GGUF file.")
print("✅ Intermediate f16 GGUF file created successfully.")

# STEP 4B: Compile the C++ tools using CMake.
print("\n--- Step 4B: Compiling C++ tools using CMake ---")
!cmake -B build
!cmake --build build --config Release

# Correct executable name based on build logs
quantize_executable_name = "llama-quantize"
quantize_executable_path = os.path.join(os.getcwd(), "build", "bin", quantize_executable_name)
if not os.path.exists(quantize_executable_path):
    raise RuntimeError(f"Failed to compile '{quantize_executable_name}'.")
print(f"✅ '{quantize_executable_name}' tool compiled successfully.")

# STEP 4C: Loop through the quantization types and create each GGUF file.
print("\n--- Step 4C: Quantizing to multiple formats ---")
for quant_type in quantization_types:
    final_gguf_path_local = f"/content/{gguf_base_name}_{quant_type.lower()}.gguf"
    print(f"\n--- Quantizing to {quant_type} ---")

    # Run the quantization command
    !{quantize_executable_path} "{intermediate_gguf_path}" "{final_gguf_path_local}" {quant_type}

    # Verify and backup each file as it's created
    if os.path.exists(final_gguf_path_local):
        file_size = os.path.getsize(final_gguf_path_local) / (1024**3)
        print(f"✅ {quant_type} GGUF created successfully! Size: {file_size:.2f} GB")

        # Backup to Google Drive
        drive_gguf_path = f"/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/{os.path.basename(final_gguf_path_local)}"
        !cp "{final_gguf_path_local}" "{drive_gguf_path}"
        print(f"   -> Backed up to: {drive_gguf_path}")
    else:
        print(f"❌ CRITICAL ERROR: Failed to create the {quant_type} GGUF file.")

# --- 5. FINAL VERIFICATION ---
%cd /content
print(f"\n\n--- FINAL SUMMARY ---")
print("All quantization tasks are complete. Verifying files in Google Drive:")
for quant_type in quantization_types:
    drive_path = f"/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/{gguf_base_name}_{quant_type.lower()}.gguf"
    if os.path.exists(drive_path):
        print(f"  - ✅ {os.path.basename(drive_path)}")
    else:
        print(f"  - ❌ {os.path.basename(drive_path)} (Missing from Drive!)")

# --- 6. FINAL CLEANUP ---
print("\nCleaning up all temporary files and directories...")
!rm -rf {local_restore_dir}
!rm -rf {merged_model_path}
!rm -rf {llama_cpp_path}
!rm -f {intermediate_gguf_path} # Remove the large intermediate file
# Also remove the local copies of the final GGUFs now that they are backed up
!rm -f /content/*.gguf
print("✅ Cleanup complete. Script finished.")

--- Mounting Google Drive ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully.

✅ Will generate 6 GGUF models: Q8_0, Q6_K, Q5_K_M, Q4_K_M, Q3_K_M, Q2_K

Restoring fine-tuned model adapters from: /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/final_model_adapters.zip
Found adapter checkpoint at: /content/restored_model_temp/content/powershell-sentinel-main/models/final_model/final_checkpoint

Loading base model in bfloat16 for a clean merge...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


✅ Base model loaded successfully.

Loading PEFT adapters and merging...
✅ Model and adapters merged successfully.

Saving the clean, high-precision merged model to: /content/merged_model_final
✅ Merged model saved and GPU memory cleaned.

Cloning llama.cpp repository and installing dependencies...
Cloning into '/content/llama.cpp'...
remote: Enumerating objects: 58777, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 58777 (delta 66), reused 23 (delta 23), pack-reused 58662 (from 4)
Receiving objects: 100% (58777/58777), 142.76 MiB | 36.06 MiB/s, done.
Resolving deltas: 100% (42596/42596), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cuda 25.6.0 requires numba<0.62.0a0,>=0.59.1, which is not installed.
cudf-cu12 25.6.0 requires numba<0.62.0a0,>=0.59.1, which is not installed.
shap

In [ ]:
# --- (Further/Stretch) 9. FURTHER PROMPT EXPERIMENT ---
import json
import os

# --- Step 1: Calculate Steps for 1 Full Epoch ---
full_clean_train_path = '/content/powershell-sentinel-main/data/generated/training_data_v0_final_clean.json'
with open(full_clean_train_path, 'r') as f:
    num_samples = len(json.load(f))

effective_batch_size = 4
steps_per_epoch = num_samples // effective_batch_size

print(f"Full clean training set has {num_samples} samples.")
print(f"Each training run will be for 1 full epoch, which is exactly {steps_per_epoch} steps.")

# --- Step 2: Define Paths and Create Directories ---
exp_dir = "models/prompt_exp_1_epoch"
drive_backup_dir = "/content/drive/MyDrive/PowerShell_Sentinel_Full_Epoch_Prompt_Exp"
results_filename = "full_epoch_prompt_results.md"
local_results_path = f"{exp_dir}/{results_filename}"
drive_results_path = f"{drive_backup_dir}/{results_filename}"

!mkdir -p {exp_dir}/prompt_c
!mkdir -p {exp_dir}/prompt_d
!mkdir -p {exp_dir}/prompt_e
!mkdir -p {drive_backup_dir}

# --- Step 3: Run Training and Backups Sequentially ---

# Run 1: Prompt C (Detailed)
print("\n--- [1/5] Starting 1-Epoch Training for Prompt C (Detailed) ---")
!python -m powershell_sentinel.train \
    --model_name meta-llama/Meta-Llama-3-8B-Instruct \
    --train_dataset scripts/prompt_engineering/dataset_prompt_C_Detailed.json \
    --preflight_train_dataset {full_clean_train_path} \
    --output_dir {exp_dir}/prompt_c \
    --test_dataset data/sets/test_set_v0.json \
    --learning_rate 2e-5 \
    --lora_rank 16 \
    --max_steps {steps_per_epoch}

print("\n--- Backing up Prompt C model to Google Drive... ---")
!cd {exp_dir} && zip -r {drive_backup_dir}/prompt_c_adapters.zip prompt_c
print("✅ Backup complete.")

# Run 2: Prompt D (Minimalist)
print("\n--- [2/5] Starting 1-Epoch Training for Prompt D (Minimalist) ---")
!python -m powershell_sentinel.train \
    --model_name meta-llama/Meta-Llama-3-8B-Instruct \
    --train_dataset scripts/prompt_engineering/dataset_prompt_D_Minimalist.json \
    --preflight_train_dataset {full_clean_train_path} \
    --output_dir {exp_dir}/prompt_d \
    --test_dataset data/sets/test_set_v0.json \
    --learning_rate 2e-5 \
    --lora_rank 16 \
    --max_steps {steps_per_epoch}

print("\n--- Backing up Prompt D model to Google Drive... ---")
!cd {exp_dir} && zip -r {drive_backup_dir}/prompt_d_adapters.zip prompt_d
print("✅ Backup complete.")

# Run 3: Prompt E (Role-Play Minimalist)
print("\n--- [3/5] Starting 1-Epoch Training for Prompt E (Role-Play Minimalist) ---")
!python -m powershell_sentinel.train \
    --model_name meta-llama/Meta-Llama-3-8B-Instruct \
    --train_dataset scripts/prompt_engineering/dataset_prompt_E_RolePlayMinimalist.json \
    --preflight_train_dataset {full_clean_train_path} \
    --output_dir {exp_dir}/prompt_e \
    --test_dataset data/sets/test_set_v0.json \
    --learning_rate 2e-5 \
    --lora_rank 16 \
    --max_steps {steps_per_epoch}

print("\n--- Backing up Prompt E model to Google Drive... ---")
!cd {exp_dir} && zip -r {drive_backup_dir}/prompt_e_adapters.zip prompt_e
print("✅ Backup complete.")

# --- Step 4: Run Final Evaluation ---
print("\n--- [4/5] Starting Final Evaluation on All 3 Models ---")
print("This will take several hours. The results will be saved and then displayed below upon completion.")
!python -m scripts.prompt_engineering.evaluate_prompts \
    --base_model_path meta-llama/Meta-Llama-3-8B-Instruct \
    --val_path scripts/prompt_engineering/mini_val.json \
    --exp_dir {exp_dir} > {local_results_path}

# --- Step 5: Backup and Display Final Results ---
print("\n--- [5/5] Backing up Final Results and Displaying ---")
!cp {local_results_path} {drive_results_path}
print(f"✅ Final results table backed up to: {drive_results_path}")

print("\n--- Final Results Table ---")
!cat {local_results_path}

print("\n\n✅ ✅ ✅ FULL PROMPT EXPERIMENT AND BACKUP COMPLETE ✅ ✅ ✅")

In [23]:
# --- 10. QUALITATIVE EVALUTION OF QUANTISED MODELS ---
# --- 1. SETUP AND DEPENDENCIES ---
print("--- Step 1: Installing Dependencies ---")
!pip install llama-cpp-python peft -q
print("✅ Dependencies installed.")

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from llama_cpp import Llama
from IPython.display import display, Markdown
import os
import gc
import json

# --- 2. CONFIGURATION AND PATHS ---
print("\n--- Step 2: Configuring Paths and Prompts ---")
# Paths for models and results
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
drive_backup_path = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/final_model_adapters.zip"
local_restore_dir = "/content/restored_model_temp"
high_precision_path = "/content/merged_model_final" # This will be recreated
gguf_base_name = "powershell_sentinel_llama3"
results_dir_drive = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_results"

# Create the results directory on Google Drive
os.makedirs(results_dir_drive, exist_ok=True)
print(f"✅ Evaluation results will be saved to: {results_dir_drive}")

# List of models to evaluate
quantization_types = ["Q8_0", "Q6_K", "Q5_K_M", "Q4_K_M", "Q3_K_M", "Q2_K"]

# Golden Dataset
evaluation_prompts = [
    "Write a PowerShell command to get the 5 most memory-intensive processes on the system.",
    "Create a PowerShell script that takes a directory path as input, zips its contents, and places the zip file on the Desktop.",
    "In PowerShell, what is the key difference between an `Array` and an `ArrayList`?",
    "Show me how to correctly add a `try-catch-finally` block to handle potential errors when trying to remove a file."
]
print(f"✅ Golden Dataset with {len(evaluation_prompts)} prompts is ready.")

# --- 3. RECREATE THE HIGH-PRECISION BASELINE MODEL ---
print("\n--- Step 3: Recreating High-Precision Baseline Model ---")
# This step is necessary because the previous script's cleanup deleted the merged model.

# A. Restore adapters
print("Restoring fine-tuned model adapters...")
!rm -rf {local_restore_dir}
!mkdir -p {local_restore_dir}
!unzip -o -q "{drive_backup_path}" -d {local_restore_dir}
adapter_path = next((os.path.join(root, d) for root, dirs, _ in os.walk(local_restore_dir) for d in dirs if "final_checkpoint" in d), None)
if not adapter_path:
    raise FileNotFoundError("Could not find 'final_checkpoint' directory.")
print(f"Found adapter checkpoint at: {adapter_path}")

# B. Load base model in bfloat16
print("\nLoading base model in bfloat16 for a clean merge...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, torch_dtype=torch.bfloat16, device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
print("✅ Base model loaded.")

# C. Merge and save
print("\nMerging PEFT adapters...")
model = PeftModel.from_pretrained(base_model, adapter_path).merge_and_unload()
print(f"Saving the clean, high-precision merged model to: {high_precision_path}")
model.save_pretrained(high_precision_path)
tokenizer.save_pretrained(high_precision_path)

# D. Clean up GPU memory
del model, base_model, tokenizer
gc.collect()
torch.cuda.empty_cache()
print("✅ Baseline model recreated and GPU memory cleaned.")


# --- 4. LOAD MODELS FOR EVALUATION ---
print("\n--- Step 4: Loading Models for Evaluation ---")
# Load the freshly recreated High-Precision Baseline Model
print("Loading high-precision baseline model...")
model_baseline = AutoModelForCausalLM.from_pretrained(
    high_precision_path, torch_dtype=torch.bfloat16, device_map="auto"
)
tokenizer_baseline = AutoTokenizer.from_pretrained(high_precision_path)
print("✅ High-precision model loaded.")


# --- 5. DEFINE GENERATION HELPERS ---
def generate_baseline_response(prompt):
    messages = [{"role": "user", "content": prompt}]
    input_ids = tokenizer_baseline.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(model_baseline.device)
    outputs = model_baseline.generate(
        input_ids, max_new_tokens=512, eos_token_id=tokenizer_baseline.eos_token_id, do_sample=False
    )
    return tokenizer_baseline.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

def generate_quantized_response(llm_instance, prompt):
    formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    output = llm_instance(formatted_prompt, max_tokens=512, echo=False, temperature=0.0)
    return output['choices'][0]['text']

print("\n✅ Generation helper functions are ready.")

# --- 6. RUN EVALUATION, DISPLAY, AND COLLECT DATA ---
print("\n--- Step 6: Starting Full Model Evaluation ---")
all_evaluation_data = []

for i, prompt in enumerate(evaluation_prompts):
    display(Markdown(f"## ➡️ Prompt {i+1}: `{prompt}`\n---"))

    prompt_results = {"prompt": prompt, "responses": {}}

    # Generate from baseline
    print(f"Generating baseline response for prompt {i+1}...")
    prompt_results["responses"]["High-Precision (f16)"] = generate_baseline_response(prompt)
    print("  -> Done.")

    # Loop through and generate from each quantized model
    for quant_type in quantization_types:
        gguf_path = f"/content/{gguf_base_name}_{quant_type.lower()}.gguf"
        model_key = f"Quantized ({quant_type})"

        if not os.path.exists(gguf_path):
            prompt_results["responses"][model_key] = "SKIPPED - FILE NOT FOUND"
            print(f"⚠️ Could not find {gguf_path}, skipping.")
            continue

        print(f"Loading and generating for {quant_type}...")
        llm_quantized = Llama(model_path=gguf_path, n_gpu_layers=-1, n_ctx=4096, verbose=False)
        prompt_results["responses"][model_key] = generate_quantized_response(llm_quantized, prompt)

        del llm_quantized
        gc.collect()
        torch.cuda.empty_cache()
        print(f"  -> Done. Unloaded {quant_type} to free memory.")

    all_evaluation_data.append(prompt_results)

    # Display comparison table
    markdown_table = "| Model                  | Generated Output |\n|------------------------|------------------|\n"
    for model_name, output in prompt_results["responses"].items():
        cleaned_output = output.replace('\n', '<br>').replace('|', '\\|')
        markdown_table += f"| **{model_name}** | <pre>{cleaned_output}</pre> |\n"
    display(Markdown(markdown_table))

print("\n--- ✅ Evaluation Loop Complete ---")

# --- 7. PERSIST RESULTS TO GOOGLE DRIVE ---
print("\n--- Step 7: Persisting Results to Drive ---")

# Save as JSON
json_path = os.path.join(results_dir_drive, "evaluation_results.json")
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(all_evaluation_data, f, indent=4)
print(f"✅ Full results saved in JSON format to: {json_path}")

# Save as Markdown
md_path = os.path.join(results_dir_drive, "evaluation_report.md")
markdown_content = "# PowerShell Sentinel - Model Quantization Evaluation Report\n\n"
for record in all_evaluation_data:
    markdown_content += f"## ➡️ Prompt: `{record['prompt']}`\n\n"
    markdown_content += "| Model                  | Generated Output |\n"
    markdown_content += "|------------------------|------------------|\n"
    for model_name, output in record['responses'].items():
        markdown_content += f"| **{model_name}** | ```powershell\n{output}\n``` |\n"
    markdown_content += "\n---\n\n"

with open(md_path, 'w', encoding='utf-8') as f:
    f.write(markdown_content)
print(f"✅ Human-readable report saved in Markdown format to: {md_path}")

# --- 8. FINAL CLEANUP ---
print("\n--- Step 8: Final Cleanup ---")
!rm -rf {local_restore_dir}
!rm -rf {high_precision_path}
print("✅ Cleaned up temporary directories.")

print("\n--- ✅✅✅ All Tasks Complete ✅✅✅ ---")

--- Step 1: Installing Dependencies ---
✅ Dependencies installed.

--- Step 2: Configuring Paths and Prompts ---
✅ Evaluation results will be saved to: /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_results
✅ Golden Dataset with 4 prompts is ready.

--- Step 3: Recreating High-Precision Baseline Model ---
Restoring fine-tuned model adapters...
Found adapter checkpoint at: /content/restored_model_temp/content/powershell-sentinel-main/models/final_model/final_checkpoint

Loading base model in bfloat16 for a clean merge...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


✅ Base model loaded.

Merging PEFT adapters...
Saving the clean, high-precision merged model to: /content/merged_model_final
✅ Baseline model recreated and GPU memory cleaned.

--- Step 4: Loading Models for Evaluation ---
Loading high-precision baseline model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

  -> Done. Unloaded Q3_K_M to free memory.
Loading and generating for Q2_K...


llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


  -> Done. Unloaded Q2_K to free memory.


| Model                  | Generated Output |
|------------------------|------------------|
| **High-Precision (f16)** | <pre>Here is an example of how to correctly add a `try-catch-finally` block to handle potential errors when trying to remove a file:<br>```<br>try {<br>    File file = new File("path/to/file.txt");<br>    if (file.exists()) {<br>        file.delete();<br>    } else {<br>        System.out.println("File does not exist");<br>    }<br>} catch (SecurityException e) {<br>    System.out.println("Error: " + e.getMessage());<br>} catch (Exception e) {<br>    System.out.println("An unexpected error occurred: " + e.getMessage());<br>} finally {<br>    System.out.println("File removal attempt complete");<br>}<br>```<br>Let me explain what's going on:<br><br>* The `try` block is where we attempt to remove the file. We first check if the file exists using the `exists()` method, and if it does, we call the `delete()` method to try to remove it.<br>* The `catch` block is where we handle any errors that might occur during the file removal attempt. We have two separate `catch` blocks:<br>	+ The first `catch` block catches `SecurityException` errors, which can occur if the program doesn't have the necessary permissions to delete the file.<br>	+ The second `catch` block catches any other type of exception that might occur, and prints a generic error message.<br>* The `finally` block is where we put any code that we want to execute regardless of whether an error occurred or not. In this case, we simply print a message indicating that the file removal attempt is complete.<br><br>By using a `try-catch-finally` block, we can ensure that our program handles potential errors in a robust way, and provides useful feedback to the user if something goes wrong.</pre> |
| **Quantized (Q8_0)** | <pre>Here is an example of how to correctly add a `try-catch-finally` block to handle potential errors when trying to remove a file:<br>```<br>try {<br>    File file = new File("path/to/file.txt");<br>    if (file.exists()) {<br>        file.delete();<br>    } else {<br>        System.out.println("File does not exist");<br>    }<br>} catch (SecurityException e) {<br>    System.out.println("Error deleting file: " + e.getMessage());<br>} finally {<br>    System.out.println("File deletion attempt complete");<br>}<br>```<br>Let me explain what's going on:<br><br>* The `try` block contains the code that may throw an exception, in this case, the `delete()` method of the `File` object.<br>* The `catch` block catches any `SecurityException` that may be thrown by the `delete()` method. You can catch other types of exceptions as well, or use a more general `Exception` catch block.<br>* The `finally` block is executed regardless of whether an exception was thrown or not. It's a good place to put any cleanup code that needs to run, such as closing resources or logging information.<br><br>In this example, if the file exists and is successfully deleted, the `finally` block will print "File deletion attempt complete". If the file does not exist, the `finally` block will still be executed. If a `SecurityException` is thrown while trying to delete the file, the `catch` block will handle it and print an error message, and the `finally` block will still be executed.<br><br>Note that you can also use a `try-with-resources` statement to automatically close the `File` object and handle any exceptions that may be thrown:<br>```<br>try (File file = new File("path/to/file.txt")) {<br>    if (file.exists()) {<br>        file.delete();<br>    } else {<br>        System.out.println("File does not exist");<br>    }<br>} catch (SecurityException e) {<br>    System.out.println("Error deleting file: " + e.getMessage());<br>} finally {<br>    System.out.println("File deletion attempt complete");<br>}<br>```<br>This is a more concise and efficient way to handle file operations, as it automatically closes the `File` object and handles any exceptions that may be thrown.</pre> |
| **Quantized (Q6_K)** | <pre>Here is an example of how to correctly add a `try-catch-finally` block to handle potential errors when trying to remove a file:<br>```<br>try {<br>    File file = new File("path/to/file.txt");<br>    if (file.exists()) {<br>        file.delete();<br>    } else {<br>        System.out.println("File does not exist");<br>    }<br>} catch (SecurityException e) {<br>    System.out.println("Error deleting file: " + e.getMessage());<br>} finally {<br>    System.out.println("File operation complete");<br>}<br>```<br>Let me explain what's going on:<br><br>* The `try` block contains the code that may throw an exception, in this case, the `delete()` method of the `File` object.<br>* The `catch` block catches any `SecurityException` that may be thrown by the `delete()` method. You can catch other types of exceptions as well, or use a more general `Exception` catch block.<br>* The `finally` block is executed regardless of whether an exception was thrown or not. It's a good place to put any cleanup code that needs to run, such as closing resources or logging information.<br><br>In this example, if the file exists and is successfully deleted, the `finally` block will print "File operation complete". If the file does not exist, the `finally` block will still be executed and print the same message. If a `SecurityException` is thrown while trying to delete the file, the `catch` block will catch it and print an error message, and the `finally` block will still be executed.<br><br>Note that you can also use a `try-with-resources` statement to automatically close the `File` object and handle any exceptions that may be thrown:<br>```<br>try (File file = new File("path/to/file.txt")) {<br>    if (file.exists()) {<br>        file.delete();<br>    } else {<br>        System.out.println("File does not exist");<br>    }<br>} catch (SecurityException e) {<br>    System.out.println("Error deleting file: " + e.getMessage());<br>} finally {<br>    System.out.println("File operation complete");<br>}<br>```<br>This is a more concise and efficient way to handle file operations, as it automatically closes the `File` object and handles any exceptions that may be thrown.</pre> |
| **Quantized (Q5_K_M)** | <pre>Here is an example of how to correctly add a `try-catch-finally` block to handle potential errors when trying to remove a file:<br>```<br>try {<br>    File file = new File("path/to/file.txt");<br>    if (file.exists()) {<br>        file.delete();<br>    } else {<br>        System.out.println("File does not exist");<br>    }<br>} catch (SecurityException e) {<br>    System.out.println("Error deleting file: " + e.getMessage());<br>} finally {<br>    System.out.println("File operation complete");<br>}<br>```<br>Let me explain what's going on:<br><br>* The `try` block contains the code that may throw an exception, in this case, the `delete()` method of the `File` object.<br>* The `catch` block catches any `SecurityException` that may be thrown by the `delete()` method. You can catch other types of exceptions as well, or use a more general `Exception` catch block.<br>* The `finally` block is executed regardless of whether an exception was thrown or not. It's a good place to put any cleanup code that needs to run, such as closing a resource or logging a message.<br><br>In this example, if the file exists and is successfully deleted, the `finally` block will print "File operation complete". If the file does not exist, the `finally` block will still be executed and print the same message. If a `SecurityException` is thrown while trying to delete the file, the `catch` block will catch it and print an error message, and the `finally` block will still be executed.<br><br>Note that you should always handle exceptions in a way that makes sense for your specific use case. In this example, I'm just printing error messages to the console, but in a real-world application, you may want to handle exceptions in a more robust way, such as by retrying the operation or logging the error.</pre> |
| **Quantized (Q4_K_M)** | <pre>Here is an example of how to correctly add a `try-catch-finally` block to handle potential errors when trying to remove a file:<br>```<br>try {<br>    File file = new File("path/to/file.txt");<br>    if (file.exists()) {<br>        file.delete();<br>    } else {<br>        System.out.println("File does not exist");<br>    }<br>} catch (SecurityException e) {<br>    System.out.println("Error: " + e.getMessage());<br>} finally {<br>    System.out.println("File operation complete");<br>}<br>```<br>Let's break down what's happening here:<br><br>* The `try` block contains the code that may throw an exception, which is the attempt to remove the file using the `delete()` method.<br>* The `catch` block catches any `SecurityException` that may be thrown by the `delete()` method. You can catch other types of exceptions as well, or use a more general `Exception` catch block if you want to catch all types of exceptions.<br>* The `finally` block is executed regardless of whether an exception was thrown or not. In this case, it simply prints a message indicating that the file operation is complete.<br><br>Note that the `finally` block is optional, but it's a good practice to include it to ensure that any resources are cleaned up or any necessary cleanup code is executed, even if an exception is thrown.<br><br>Also, it's a good idea to check if the file exists before trying to delete it, as shown in the example. This can help prevent a `FileNotFoundException` from being thrown.<br><br>You can also use the `File.delete()` method in a more concise way using the `Optional` class:<br>```<br>Optional<File> file = Optional.ofNullable(new File("path/to/file.txt"));<br>file.ifPresent(f -> f.delete());<br>```<br>This will delete the file if it exists, and do nothing if it doesn't exist.</pre> |
| **Quantized (Q3_K_M)** | <pre>Here is an example of how to correctly add a `try-catch-finally` block to handle potential errors when trying to remove a file:<br>```<br>try {<br>    if (File.Exists("path/to/file.txt")) {<br>        File.Delete("path/to/file.txt");<br>    }<br>} catch (IOException ex) {<br>    Console.WriteLine("Error deleting file: " + ex.Message);<br>} finally {<br>    // This block will always be executed, regardless of whether an exception was thrown<br>    // You can put any cleanup code here, such as closing a file stream<br>}<br>```<br>In this example, we're trying to delete a file at the specified path. We use the `File.Exists` method to check if the file exists before attempting to delete it. If the file exists, we use the `File.Delete` method to try to delete it.<br><br>The `try` block contains the code that may throw an exception. In this case, the `File.Delete` method may throw an `IOException` if there's an error deleting the file.<br><br>The `catch` block catches any `IOException` exceptions that are thrown by the `File.Delete` method. We can handle the exception by logging an error message to the console, for example.<br><br>The `finally` block is executed regardless of whether an exception was thrown or not. This is a good place to put any cleanup code, such as closing a file stream.<br><br>Note that you can also use a `using` statement to ensure that a file stream is properly closed, regardless of whether an exception is thrown:<br>```<br>using (var fileStream = File.Open("path/to/file.txt", FileMode.Open))<br>{<br>    fileStream.Dispose();<br>}<br>```<br>This is a more modern and recommended way to handle file I/O operations in C#.</pre> |
| **Quantized (Q2_K)** | <pre>Here is an example of how you can add a `try-catch-finally` block to handle potential errors when trying to remove a file:<br>```<br>try {<br>    // Code that might throw an exception<br>    File file = new File("path/to/file.txt");<br>    file.delete();<br>} catch (IOException e) {<br>    // Error handling code<br>    System.out.println("Error removing file: " + e.getMessage());<br>} finally {<br>    // Code that will always run, regardless of whether an exception was thrown<br>    System.out.println("File removal attempt complete");<br>}<br>```<br>In this example, the `try` block contains the code that might throw an exception when trying to remove the file. The `catch` block handles the error, printing an error message to the console. The `finally` block is used to execute code that will always run, regardless of whether an exception was thrown.<br><br>Here's a breakdown of the different parts:<br><br>* `try`: This is the code that might throw an exception. In this case, it's the file deletion operation.<br>* `catch`: This is the error handling code. It catches the exception thrown by the `try` block and prints an error message to the console.<br>* `finally`: This is the code that will always run, regardless of whether an exception was thrown. In this case, it's a message indicating that the file removal attempt has completed.<br><br>Note that the `finally` block is optional, but it's a good practice to include it to ensure that you always execute some code after the try-catch block.<br><br>Also, make sure to handle the exception properly, as it's a good practice to handle exceptions in a way that makes sense for your application.</pre> |



--- ✅ Evaluation Loop Complete ---

--- Step 7: Persisting Results to Drive ---
✅ Full results saved in JSON format to: /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_results/evaluation_results.json
✅ Human-readable report saved in Markdown format to: /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_results/evaluation_report.md

--- Step 8: Final Cleanup ---
✅ Cleaned up temporary directories.

--- ✅✅✅ All Tasks Complete ✅✅✅ ---


In [ ]:
# --- (FURTHER/STRETCH) 11. QUANTITATIVE EVALUATION OF QUANTISED MODELS ---
# ==============================================================================
# BLOCK 1: QUANTITATIVE EVALUATION - SMOKE TEST (Definitive Version)
# Purpose: A quick run on a small subset of data to verify the entire pipeline.
# ==============================================================================

# --- 1. SETUP ---
print("--- Step 1: Building a Stable Environment ---")
# THE DEFINITIVE FIX: Manually uninstall conflicting packages and then install
# specific, known-compatible versions to prevent ABI conflicts.

# Force uninstall the default libraries to ensure a clean slate.
!pip uninstall -y numpy scipy scikit-learn pandas numba
print("\n[Phase 1/3] Uninstalled conflicting base packages.")

# Reinstall specific, stable versions that are known to be compatible.
# We are pinning numpy to a version before the major 2.0 release.
!pip install numpy==1.26.4
print("\n[Phase 2/3] Installed a stable version of NumPy.")

# Now, install the rest of the stack, which will use the stable numpy.
!pip install llama-cpp-python scikit-learn pandas -q
print("\n[Phase 3/3] Installed remaining libraries.")
print("✅ Dependencies successfully rebuilt in a stable configuration.")


# The Colab runtime needs to be restarted to load the new library versions correctly.
# This will crash the current execution, which is NORMAL AND EXPECTED.
# After it crashes, simply run this SAME CELL AGAIN from the top. The second
# time, it will load the correct libraries and proceed.
print("\n\n\033[1m\033[91m--> IMPORTANT: The Colab runtime is now restarting to load the correct libraries. <--"
      "\nAfter the restart is complete, please run this cell again to proceed with the smoke test.\033[0m")
import os
os.kill(os.getpid(), 9)


# --- The rest of the script is only executed on the SECOND run after the restart ---
import json
import gc
import torch
from llama_cpp import Llama
import pandas as pd
from sklearn.metrics import classification_report
from IPython.display import display, Markdown

# --- 2. CONFIGURATION ---
print("\n--- Step 2: Configuring Paths for Smoke Test ---")
test_set_path = "/content/data/sets/test_set_v0.json"
gguf_base_name = "powershell_sentinel_llama3"
results_dir_drive = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_results"
os.makedirs(results_dir_drive, exist_ok=True)
quantization_types = ["Q8_0", "Q6_K", "Q5_K_M", "Q4_K_M", "Q3_K_M", "Q2_K"]

# --- 3. LOAD TEST DATA ---
print(f"\n--- Step 3: Loading Test Data from {test_set_path} ---")
if not os.path.exists(test_set_path):
    raise FileNotFoundError(f"CRITICAL: The test set file was not found at {test_set_path}.")
with open(test_set_path, 'r') as f:
    test_data = json.load(f)
smoke_test_data = test_data[:15]
print(f"✅ Loaded {len(test_data)} total records. 🔥 SMOKE TEST will run on {len(smoke_test_data)} records.")

# --- 4. RUN QUANTITATIVE EVALUATION ---
print("\n--- Step 4: Starting Quantitative SMOKE TEST Loop ---")
final_results_smoke = []

for quant_type in quantization_types:
    gguf_path = f"/content/{gguf_base_name}_{quant_type.lower()}.gguf"
    if not os.path.exists(gguf_path):
        print(f"⚠️ SKIPPING {quant_type} - GGUF file not found at {gguf_path}.")
        continue

    print(f"\n--- Evaluating Model: {quant_type} ---")
    llm = Llama(model_path=gguf_path, n_gpu_layers=-1, n_ctx=4096, verbose=False)

    parse_success_count = 0
    ground_truths_intent = []
    predictions_intent = []

    for item in smoke_test_data:
        obfuscated_command = item['input']
        ground_truth_json = item['output']
        formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nAnalyze the following PowerShell command and provide the output in a structured JSON format:\n\n```powershell\n{obfuscated_command}\n```<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        output = llm(formatted_prompt, max_tokens=1024, echo=False, temperature=0.0)
        response_text = output['choices'][0]['text']

        try:
            json_block = response_text[response_text.find('{') : response_text.rfind('}')+1]
            parsed_json = json.loads(json_block)
            parse_success_count += 1
            ground_truths_intent.append(ground_truth_json.get("intent", "N/A"))
            predictions_intent.append(parsed_json.get("intent", "Error"))
        except (json.JSONDecodeError, IndexError):
            ground_truths_intent.append(ground_truth_json.get("intent", "N/A"))
            predictions_intent.append("FailedToParse")

    parse_rate = (parse_success_count / len(smoke_test_data)) * 100
    report = classification_report(ground_truths_intent, predictions_intent, output_dict=True, zero_division=0)

    model_stats = {
        "model": quant_type,
        "parse_success_rate": f"{parse_rate:.2f}%",
        "intent_f1_score": f"{report['weighted avg']['f1-score']:.2f}",
    }
    final_results_smoke.append(model_stats)
    print(f"✅ {quant_type} (Smoke Test): Parse Rate = {parse_rate:.2f}%, Intent F1 = {report['weighted avg']['f1-score']:.2f}")

    del llm
    gc.collect()
    torch.cuda.empty_cache()

# --- 5. DISPLAY SMOKE TEST RESULTS ---
print("\n--- Step 5: Smoke Test Summary ---")
df_smoke = pd.DataFrame(final_results_smoke)
display(Markdown("### 🔥 Smoke Test Performance Summary"))
display(df_smoke)
print("\n✅ Smoke test complete. If the table above looks reasonable, you are clear to run the full evaluation script.")

--- Step 1: Building a Stable Environment ---
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4

[Phase 1/3] Uninstalled conflicting base packages.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.1 requires pandas, which is not installed.
seaborn 0.13.2 requires pandas>=1.2, which is not installed.
pymc 5.25.1 requires pandas>=0.24.0, which is not installed.
pymc 5.25.1 requires scipy>=1.4.1, which is not installed.
bigframes 2.13.0 requires pandas>=1.5.3, which is not installed.
mlxtend 0.23.4 requires pandas>=0.24.2, which is not installed.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, which i


[Phase 2/3] Installed a stable version of NumPy.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires numba>=0.51.2, which is not installed.
librosa 0.11.0 requires numba>=0.51.0, which is not installed.
dask-cuda 25.6.0 requires numba<0.62.0a0,>=0.59.1, which is not installed.
stumpy 1.13.0 requires numba>=0.57.1, which is not installed.
cudf-cu12 25.6.0 requires numba<0.62.0a0,>=0.59.1, which is not installed.
shap 0.48.0 requires numba>=0.54, which is not installed.
pynndescent 0.5.13 requires numba>=0.51.2, which is not installed.
cuml-cu12 25.6.0 requires numba<0.62.0a0,>=0.59.1, which is not installed.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
gradio 5.41.0 requires huggingface-hub<1.0,>=0.33.5, but you have huggingface-hub 0.23.5 which is incompatible.
cudf-cu12 25.6.0 requires

In [26]:
# --- (FURTHER/STRETCH) 11. QUANTITATIVE EVALUATION OF QUANTISED MODELS ---
# ==============================================================================
# BLOCK 2: QUANTITATIVE EVALUATION - FULL RUN
# Purpose: The definitive, long-running evaluation on the entire test set.
# ==============================================================================

# --- 1. SETUP ---
print("--- Step 1: Installing Dependencies for Full Evaluation ---")
# Dependencies should already be installed from the smoke test, but this ensures it.
!pip install llama-cpp-python scikit-learn pandas -q
print("✅ Dependencies installed.")

import json
import os
import gc
import torch
from llama_cpp import Llama
import pandas as pd
from sklearn.metrics import classification_report
from IPython.display import display, Markdown

# --- 2. CONFIGURATION ---
print("\n--- Step 2: Configuring Paths ---")
# This is the "locked" test set the model has never seen
test_set_path = "/content/data/sets/test_set_v0.json"
gguf_base_name = "powershell_sentinel_llama3"
results_dir_drive = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_results"
os.makedirs(results_dir_drive, exist_ok=True)

# List of models to evaluate
quantization_types = ["Q8_0", "Q6_K", "Q5_K_M", "Q4_K_M", "Q3_K_M", "Q2_K"]

# --- 3. LOAD TEST DATA ---
print(f"\n--- Step 3: Loading Full Test Data from {test_set_path} ---")
if not os.path.exists(test_set_path):
    raise FileNotFoundError(f"CRITICAL: The test set file was not found at {test_set_path}.")

with open(test_set_path, 'r') as f:
    test_data = json.load(f)
print(f"✅ Loaded {len(test_data)} records for the full evaluation.")

# --- 4. RUN QUANTITATIVE EVALUATION ---
print("\n--- Step 4: Starting FULL Quantitative Evaluation Loop ---")

final_results = []

# Loop through each quantized model
for quant_type in quantization_types:
    gguf_path = f"/content/{gguf_base_name}_{quant_type.lower()}.gguf"
    if not os.path.exists(gguf_path):
        print(f"⚠️ SKIPPING {quant_type} - GGUF file not found at {gguf_path}.")
        continue

    print(f"\n--- Evaluating Model: {quant_type} ---")
    llm = Llama(model_path=gguf_path, n_gpu_layers=-1, n_ctx=4096, verbose=False)

    parse_success_count = 0
    ground_truths_intent = []
    predictions_intent = []

    # Loop through each item in the FULL test data
    for i, item in enumerate(test_data):
        if (i + 1) % 50 == 0:
            print(f"  -> Processing record {i+1} of {len(test_data)}...")

        obfuscated_command = item['input']
        ground_truth_json = item['output']

        formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nAnalyze the following PowerShell command and provide the output in a structured JSON format:\n\n```powershell\n{obfuscated_command}\n```<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

        output = llm(formatted_prompt, max_tokens=1024, echo=False, temperature=0.0)
        response_text = output['choices'][0]['text']

        # Layer 2: Test JSON Parse Success
        try:
            json_block = response_text[response_text.find('{') : response_text.rfind('}')+1]
            parsed_json = json.loads(json_block)
            parse_success_count += 1

            # Layer 3: Collect data for F1-Score (Intent field)
            ground_truths_intent.append(ground_truth_json.get("intent", "N/A"))
            predictions_intent.append(parsed_json.get("intent", "Error"))
        except (json.JSONDecodeError, IndexError):
            ground_truths_intent.append(ground_truth_json.get("intent", "N/A"))
            predictions_intent.append("FailedToParse")

    # Calculate final metrics for this model
    parse_rate = (parse_success_count / len(test_data)) * 100
    report = classification_report(ground_truths_intent, predictions_intent, output_dict=True, zero_division=0)

    model_stats = {
        "model": quant_type,
        "parse_success_rate": f"{parse_rate:.2f}%",
        "intent_f1_score": f"{report['weighted avg']['f1-score']:.2f}",
        "full_classification_report": report
    }
    final_results.append(model_stats)

    print(f"✅ {quant_type} (Full Run): Parse Rate = {parse_rate:.2f}%, Intent F1 = {report['weighted avg']['f1-score']:.2f}")

    del llm
    gc.collect()
    torch.cuda.empty_cache()

# --- 5. PERSIST QUANTITATIVE RESULTS ---
print("\n--- Step 5: Persisting Quantitative Results to Drive ---")
# Save raw results to JSON for deep analysis
json_path = os.path.join(results_dir_drive, "quantitative_results_full.json")
with open(json_path, 'w') as f:
    json.dump(final_results, f, indent=4)
print(f"✅ Full quantitative results (with classification reports) saved to: {json_path}")

# Create and display a clean summary table with Pandas
df_summary = pd.DataFrame(final_results)[["model", "parse_success_rate", "intent_f1_score"]]
display(Markdown("### 📊 Final Performance Summary"))
display(df_summary)

# Save the summary table to a Markdown file for the dissertation
md_report_path = os.path.join(results_dir_drive, "quantitative_summary_report.md")
df_summary.to_markdown(md_report_path, index=False)
print(f"✅ Quantitative summary table saved to: {md_report_path}")

print("\n--- ✅✅✅ All Tasks Complete ✅✅✅ ---")

--- Step 1: Installing Dependencies for Full Evaluation ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package

ModuleNotFoundError: No module named 'numpy.rec'